In [1]:
# -*- coding:gb2312 -*-  # 指定文件编码格式为GB2312，支持中文注释
import copy  # 导入深拷贝模块，用于生成独立的对象副本

def init_pass(T):
    """初始化第一轮候选1-项集并计算支持度"""
    C = {}  # 创建空字典存储项集支持度
    for t in T:  # 遍历每个事务
        for item in t:  # 遍历事务中的每个项
            C[item] = C.get(item, 0) + 1  # 使用get方法实现计数器初始化与累加,不存在则返回 0，存在则返回当前值
    return C  # 返回格式：{项: 支持度计数}

def generate(F):
    """生成下一轮的候选项集（Apriori-gen算法）"""
    C = []  # 存储生成的候选项
    k = len(F[0]) + 1  # 计算新候选项维度（当前项集长度+1）
    
    # 双重循环遍历所有可能的项集对
    for i in range(len(F)):  # 外层循环索引
        for j in range(i+1, len(F)):  # 内层循环避免重复组合
            f1, f2 = F[i], F[j]  # 获取两个待合并的项集
            
            # 合并条件：前k-2项相同且末项有序（避免重复）
            if f1[:-1] == f2[:-1] and f1[-1] < f2[-1]:
                new_item = f1.copy()  # 深拷贝防止修改原数据
                new_item.append(f2[-1])  # 合并生成k项集
                
                # 剪枝：验证所有k-1子集是否都在F中
                valid = True
                for idx in range(len(new_item)):  # 遍历每个可能删除的位置
                    subset = new_item[:idx] + new_item[idx+1:]  # 生成k-1项子集
                    if subset not in F:  # 若子集不在当前频繁集中则剪枝
                        valid = False
                        break
                # 通过验证且未重复则加入候选集
                if valid and new_item not in C:
                    C.append(new_item)
    return C  # 返回格式：[[项1,项2,...], ...]

def is_subset(candidate, transaction):
    """正确判断候选项是否是事务的子集"""
    # 使用生成器表达式高效判断包含关系
    return all(item in transaction for item in candidate)

def apriori(T, min_sup):
    """Apriori算法主函数"""
    all_frequent = []  # 存储所有找到的频繁项集
    
    # === 第一轮处理：频繁1-项集 ===
    C1 = init_pass(T)  # 获取候选1-项集支持度
    # 筛选满足最小支持度的项（注意C1是字典结构）
    F1 = [[item] for item, cnt in C1.items() if cnt >= min_sup]
    F1.sort()  # 按字母顺序排序保证后续合并正确性
    all_frequent.extend(F1)  # 存入结果集
    
    # === 迭代处理后续轮次 ===
    F_prev = F1  # 初始化前次频繁项集
    k = 2  # 当前处理项集维度
    while F_prev:  # 当存在频繁项集时继续
        # -- 候选集生成阶段 --
        Ck = generate(F_prev)  # 调用Apriori-gen生成候选项
        
        # -- 支持度计算阶段 --
        candidate_counts = {}  # 使用字典统计支持度
        for candidate in Ck:  # 遍历每个候选项
            for transaction in T:  # 扫描所有事务
                if is_subset(candidate, transaction):
                    # 列表不可哈希，转为元组作为字典键
                    key = tuple(candidate)
                    candidate_counts[key] = candidate_counts.get(key, 0) + 1
        
        # -- 频繁项筛选阶段 --
        # 转换回列表结构并筛选
        Fk = [list(key) for key, cnt in candidate_counts.items() if cnt >= min_sup]
        Fk.sort()  # 保持项集有序（重要！影响后续合并）
        
        if Fk:  # 如果找到新频繁项集
            all_frequent.extend(Fk)  # 存入总结果
            F_prev = Fk  # 更新前次频繁项集
            k += 1  # 增加维度
        else:  # 终止条件：无新频繁项
            F_prev = None
    
    return all_frequent  # 返回所有找到的频繁项集

# === 测试执行 ===
# 定义测试数据集（4个事务）
T = [['A','C','D'],
     ['B','C','E'],
     ['A','B','C','E'],
     ['B','E']]

# 执行算法（最小支持度计数=2）
result = apriori(T, 2)

# 输出结果
print("频繁项集：")
for itemset in result:
    # 将项集转换为字母排序字符串输出
    print(sorted(itemset))  # 示例输出：['B','C','E']

频繁项集：
['A']
['B']
['C']
['E']
['A', 'C']
['B', 'C']
['B', 'E']
['C', 'E']
['B', 'C', 'E']
